크롤링

In [ ]:
import selenium #셀레니움
import pandas as pd #csv를 읽고 dataframe을 사용하기 위한 pandas
from selenium import webdriver #브라우저를 띄우고 컨트롤하기 위한 webdriver
from selenium.webdriver.common.keys import Keys #브라우저에 키입력 용
from selenium.webdriver.common.by import By #webdriver를 이용해 태그를 찾기 위함
from selenium.webdriver.support.ui import WebDriverWait #Explicitly wait을 위함
from webdriver_manager.chrome import ChromeDriverManager #크롬에서 크롤링 진행 크롬 웹 드라이버 설치시 필요
from selenium.webdriver.support import expected_conditions as EC #브라우저에 특정 요소 상태 확인을 위해
from bs4 import BeautifulSoup #브라우저 태그를 가져오고 파싱하기 위함
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException,TimeoutException #예외처리를 위한 예외들
from time import sleep
#크롬 드라이버 최신 버전 https://googlechromelabs.github.io/chrome-for-testing/

In [ ]:
##최종##
store_name_lst = []
category_lst = []
phone_num_lst = []
address_lst = []
score_lst = []
visitor_review_lst = []

def switch_left():
    ############## iframe으로l 왼쪽 포커스 맞추기 ##############
    driver.switch_to.parent_frame()
    iframe = driver.find_element(By.XPATH,'//*[@id="searchIframe"]')
    driver.switch_to.frame(iframe)
    
def switch_right():
    ############## iframe으로 오른쪽 포커스 맞추기 ##############
    driver.switch_to.parent_frame()
    #가게명 클릭 후 우측 프레임 로딩 대기
    WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.ID, "entryIframe"))
    )
    sleep(0.2) #혹시 몰라 추가시간

    iframe = driver.find_element(By.XPATH,'//*[@id="entryIframe"]')
    driver.switch_to.frame(iframe)


options = webdriver.ChromeOptions()
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3')
options.add_argument('window-size=1380,900')
#driver = webdriver.Chrome(ChromeDriverManager().install()) # 크롬 드라이버 설치하지 않은 경우
driver = webdriver.Chrome() # 크롬 드라이버 설치한 경우
 
# 대기 시간
driver.implicitly_wait(time_to_wait=10)
 
# 반복 종료 조건
loop = True

driver.get("https://map.naver.com/v5/") #네이버지도 열기
try:
   element = WebDriverWait(driver, 10).until(
       EC.presence_of_element_located((By.CLASS_NAME, "input_search"))
   ) #입력창이 뜰 때까지 대기
finally:
   pass

keyword = str(input('키워드를 입력하시오: '))

search_box = driver.find_element(By.CLASS_NAME,"input_search") #검색창 찾기
search_box.send_keys(keyword) #검색어 입력
search_box.send_keys(Keys.ENTER) #검색

 
driver.implicitly_wait(10)

while(loop):
    switch_left()

    ############## 맨 밑까지 스크롤 ##############
    scrollable_element = driver.find_element(By.CLASS_NAME, "Ryr1F")
    last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_element)


    while True:
        # 요소 내에서 아래로 600px 스크롤
        driver.execute_script("arguments[0].scrollTop += 1500;", scrollable_element)
 
        # 페이지 로드를 기다림
        sleep(1)  # 동적 콘텐츠 로드 시간에 따라 조절
 
        # 새 높이 계산
        new_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_element)
 
        # 스크롤이 더 이상 늘어나지 않으면 루프 종료
        if new_height == last_height:
            break
 
        last_height = new_height
 

    ############## 현재 page number 가져오기 ##############
 
    page_no = driver.find_element(By.XPATH,'//a[contains(@class, "mBN2s qxokY")]').text
    
    # 현재 페이지에 등록된 모든 가게 조회
    # 첫페이지 광고 2개 때문에 첫페이지는 앞 2개를 빼야함

    elements = driver.find_elements(By.XPATH,'//*[@id = "_pcmap_list_scroll_container"]/ul/li') #광고 포함 페이지 전체 elements

    print('현재 ' + '\033[95m' + str(page_no) + '\033[0m' + ' 페이지 / '+ '총 ' + '\033[95m' + str(len(elements)) + '\033[0m' + '개(광고 포함)의 가게를 찾았습니다.\n')

    '''
    for index, e in enumerate(elemets, start=1):
        try:#광고인 경우
            ad = e.find_element(By.CLASS_NAME,'gU6bV').find_element(By.XPATH, ".//span") 
            continue #다음 for문 루프
        except:
            ####final_element = e.find_element(By.CLASS_NAME,'CHC5F').find_element(By.XPATH, ".//a/div/div/span[1]")
            elemets.append(e) #논광고만 삽입
            
        #####print(str(index) + ". " + final_element.text)


    '''
    switch_left()
 
    
    for index, e in enumerate(elements, start=1): #기존: elemets
        store_name = '' # 가게 이름
        category = '' # 카테고리
        address = '' # 가게 주소
        business_hours = [] # 영업 시간
        phone_num = '' # 전화번호
 
        switch_left()
        '''가게명 클릭 '''   
        #우측 그림 or 없음 - type1
        if e.get_attribute("data-laim-exp-id")=="undefined": 
            e.find_elements(By.XPATH, "./div[1]/div")[-1].find_element(By.XPATH,"./a[1]/div/div/span[1]").click() 

        #아래에 그림있는 경우 - type2
        elif e.get_attribute("data-laim-exp-id")=="undefinedundefined":
            e.find_element(By.CLASS_NAME,'CHC5F').find_element(By.XPATH, "./a/div/div/span[1]").click() 
              
        #광고 - type3
        else:  #"undefined*e"
            continue #다음 for문 루프로
        ''''''
        # button = driver.find_element_by_xpath(".//a/div/div/span")
        # driver.execute_script("arguments[0].click();", button)


        switch_right()
 
        ##### 상세정보 크롤링 ####
        title = driver.find_element(By.XPATH,'//div[@class = "zD5Nm undefined"]')
        # store_info = title.find_elements(By.XPATH,'//div[@class="YouOG DZucB"]/div/span')
 
        # 가게 이름
        store_name = title.find_element(By.XPATH,'.//div[1]/div[1]/span[1]').text
        # 카테고리
        category = title.find_elements(By.XPATH,'.//div[1]/div[1]/span')[-1].text
        #전화번호
        try:
            phone_num = driver.find_element(By.XPATH,'//span[@class = "xlx7Q"]').text
            driver.implicitly_wait(1) #전화번호를 0.5초 안에 못찾으면, 원래 없다고 간주
        except:
            phone_num = '-'
  
        # 가게 주소
        address = driver.find_element(By.XPATH,'//span[@class = "LDgIH"]').text

        #리뷰 수
        
        try:
            reviews = title.find_elements(By.XPATH, './/div[@class = "dAsGb"]/span[@class = "PXMot"]') #[방문자, 블로그]
            visitor_review = ''
            for review in reviews:
                v_review = review.find_element(By.XPATH,'./a').text
                visitor_review += v_review + ' '
                
            #visitor_review = [review.find_element(By.XPATH,'./a').text for review in reviews]
        

            driver.implicitly_wait(1) #0.1초 안에 못찾으면, 없는 것으로 간주
        except:
            visitor_review = '-'


        #별점
        try:

            score = title.find_element(By.XPATH, './/div[@class = "dAsGb"]/span[@class = "PXMot LXIwF"]').text

            driver.implicitly_wait(0.5) #0.1초 안에 못찾으면, 없는 것으로 간주
        except:
            score = '-' 


        print(f'가게명: {store_name},   업종: {category},   전화번호: {phone_num},   주소: {address},   별점: {score}   리뷰: {visitor_review}')
        store_name_lst.append(store_name)
        category_lst.append(category)
        phone_num_lst.append(phone_num)
        address_lst.append(address)
        score_lst.append(score)
        visitor_review_lst.append(visitor_review)
    
    switch_left()

    # 이건 페이지 넘어갈때마다 확인 -> true라면 마지막 페이지임
    next_page = driver.find_elements(By.XPATH,'//*[@id="app-root"]/div/div[2]/div[2]/a')[-1]

    # 페이지 다음 버튼이 활성화 상태일 경우 계속 진행
    if(next_page.get_attribute('aria-disabled') == 'false'):
        driver.find_elements(By.XPATH,'//*[@id="app-root"]/div/div[2]/div[2]/a')[-1].click()
    # 아닐 경우 루프 정지
    else:
        loop = False #모두 끝

키워드를 입력하시오: 익산시 수영장
현재 1 페이지 / 총 5개(광고 포함)의 가게를 찾았습니다.

가게명: 익산교육문화회관수영장,   업종: 수영장,   전화번호: 063-840-7200,   주소: 전북 익산시 고봉로 55 익산교육문화회관수영장,   별점: -   리뷰: 방문자리뷰 4 블로그리뷰 8 
가게명: 키즈풀,   업종: 수영장,   전화번호: 063-852-7889,   주소: 전북 익산시 고봉로18길 28,   별점: 별점
4.49   리뷰: 방문자리뷰 74 블로그리뷰 7 
가게명: 실내잠수전용풀장,   업종: 수영장,   전화번호: -,   주소: 전북 익산시 미륵사지로4길 30,   별점: -   리뷰: 블로그리뷰 1 
가게명: 용머리고을 물놀이장,   업종: 수영장,   전화번호: -,   주소: 전북 익산시 강변로 1087-12,   별점: -   리뷰: 블로그리뷰 3 
가게명: 강경실내수영장,   업종: 수영장,   전화번호: -,   주소: 충남 논산시 동안로112번길 46,   별점: -   리뷰: 방문자리뷰 3 블로그리뷰 4 


In [ ]:
# store_name_lst, category_lst, phone_num_lst, address_lst, score_lst, visitor_review_lst

df = pd.DataFrame(zip(store_name_lst, category_lst, phone_num_lst, address_lst, score_lst, visitor_review_lst))
df.columns = ['가게명', '업종', '전화번호', '주소', '별점', '리뷰']
df

,가게명,업종,전화번호,주소,별점,리뷰
0,익산교육문화회관수영장,수영장,063-840-7200,전북 익산시 고봉로 55 익산교육문화회관수영장,-,방문자리뷰 4 블로그리뷰 8
1,키즈풀,수영장,063-852-7889,전북 익산시 고봉로18길 28,별점\n4.49,방문자리뷰 74 블로그리뷰 7
2,실내잠수전용풀장,수영장,-,전북 익산시 미륵사지로4길 30,-,블로그리뷰 1
3,용머리고을 물놀이장,수영장,-,전북 익산시 강변로 1087-12,-,블로그리뷰 3
4,강경실내수영장,수영장,-,충남 논산시 동안로112번길 46,-,방문자리뷰 3 블로그리뷰 4


In [ ]:
df_no_duplicates = df.drop_duplicates(subset=df.columns)
df_no_duplicates

,가게명,업종,전화번호,주소,별점,리뷰
0,익산교육문화회관수영장,수영장,063-840-7200,전북 익산시 고봉로 55 익산교육문화회관수영장,-,방문자리뷰 4 블로그리뷰 8
1,키즈풀,수영장,063-852-7889,전북 익산시 고봉로18길 28,별점\n4.49,방문자리뷰 74 블로그리뷰 7
2,실내잠수전용풀장,수영장,-,전북 익산시 미륵사지로4길 30,-,블로그리뷰 1
3,용머리고을 물놀이장,수영장,-,전북 익산시 강변로 1087-12,-,블로그리뷰 3
4,강경실내수영장,수영장,-,충남 논산시 동안로112번길 46,-,방문자리뷰 3 블로그리뷰 4


In [ ]:
#주소 간소화시켜주는 함수(ex) 명달로 9길)
def get_addrs(x):
    x1 = x.split(' ')
    return " ".join(x1[2:4])

df_no_duplicates['주소2'] = df['주소'].apply(get_addrs)

In [ ]:
df_no_duplicates

,가게명,업종,전화번호,주소,별점,리뷰,주소2
0,익산교육문화회관수영장,수영장,063-840-7200,전북 익산시 고봉로 55 익산교육문화회관수영장,-,방문자리뷰 4 블로그리뷰 8,고봉로 55
1,키즈풀,수영장,063-852-7889,전북 익산시 고봉로18길 28,별점\n4.49,방문자리뷰 74 블로그리뷰 7,고봉로18길 28
2,실내잠수전용풀장,수영장,-,전북 익산시 미륵사지로4길 30,-,블로그리뷰 1,미륵사지로4길 30
3,용머리고을 물놀이장,수영장,-,전북 익산시 강변로 1087-12,-,블로그리뷰 3,강변로 1087-12
4,강경실내수영장,수영장,-,충남 논산시 동안로112번길 46,-,방문자리뷰 3 블로그리뷰 4,동안로112번길 46


네이버지도 API 활용

In [ ]:
import pandas as pd
import requests

#위경도 변환 함수
def get_lat_lng(address):
    client_id = 's8pego1ogd' #각자 네이버 api 받은것
    client_secret = 'IDwfqb34hdFls7BxU0V8WknUzJ0HrQn46NUjfBn9'
    
    url = f'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query={address}'
    headers = {
        'X-NCP-APIGW-API-KEY-ID': client_id,
        'X-NCP-APIGW-API-KEY': client_secret
    }
    
    response = requests.get(url, headers=headers)
    data = response.json()
    
    if 'addresses' in data:
        if len(data['addresses']) > 0:
            latitude = data['addresses'][0]['y']
            longitude = data['addresses'][0]['x']
            return latitude, longitude
        else:
            return None, None
    else:
        return None, None

def convert_address_to_lat_lng(df, address_column):
    latitudes = []
    longitudes = []
    for index, row in df.iterrows():
        address = row[address_column]
        if address:
            latitude, longitude = get_lat_lng(address)
            latitudes.append(latitude)
            longitudes.append(longitude)
        else:
            latitudes.append(None)
            longitudes.append(None)
    df['Latitude'] = latitudes
    df['Longitude'] = longitudes


# 주소를 위도와 경도로 변환하여 데이터프레임에 추가
convert_address_to_lat_lng(df_no_duplicates, '주소2')

In [ ]:
df_no_duplicates

,가게명,업종,전화번호,주소,별점,리뷰,주소2,Latitude,Longitude
0,익산교육문화회관수영장,수영장,063-840-7200,전북 익산시 고봉로 55 익산교육문화회관수영장,-,방문자리뷰 4 블로그리뷰 8,고봉로 55,35.9389703,126.9606669
1,키즈풀,수영장,063-852-7889,전북 익산시 고봉로18길 28,별점\n4.49,방문자리뷰 74 블로그리뷰 7,고봉로18길 28,35.9522527,126.9719301
2,실내잠수전용풀장,수영장,-,전북 익산시 미륵사지로4길 30,-,블로그리뷰 1,미륵사지로4길 30,35.9909921,127.0458751
3,용머리고을 물놀이장,수영장,-,전북 익산시 강변로 1087-12,-,블로그리뷰 3,강변로 1087-12,36.1285263,126.9316093
4,강경실내수영장,수영장,-,충남 논산시 동안로112번길 46,-,방문자리뷰 3 블로그리뷰 4,동안로112번길 46,36.1468268,127.0228360


In [ ]:
df_no_duplicates.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5 entries, 0 to 4
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   가게명        5 non-null      object
 1   업종         5 non-null      object
 2   전화번호       5 non-null      object
 3   주소         5 non-null      object
 4   별점         5 non-null      object
 5   리뷰         5 non-null      object
 6   주소2        5 non-null      object
 7   Latitude   5 non-null      object
 8   Longitude  5 non-null      object
dtypes: object(9)
memory usage: 400.0+ bytes


In [ ]:
#float 타입으로 변경
df_no_duplicates['Latitude'] = df_no_duplicates['Latitude'].astype('float')
df_no_duplicates['Longitude'] = df_no_duplicates['Longitude'].astype('float')

In [ ]:
for i in range(len(df_no_duplicates)):
    lat = df_no_duplicates.loc[i, 'Latitude']
    long = df_no_duplicates.loc[i, 'Longitude']
    name = df_no_duplicates.loc[i,'가게명']


    print(name, lat, long)

익산교육문화회관수영장 35.9389703 126.9606669
키즈풀 35.9522527 126.9719301
실내잠수전용풀장 35.9909921 127.0458751
용머리고을 물놀이장 36.1285263 126.9316093
강경실내수영장 36.1468268 127.022836


In [ ]:
import folium

#지도를 표시할 때 위도와 경도의 평균을 중심으로 보여준다!
lat_m = df_no_duplicates['Latitude'].mean()
long_m = df_no_duplicates['Longitude'].mean()

lat_m

m = folium.Map(location=[lat_m,long_m],
               zoom_start=12)
m

#데이터프레임을 처음부터 순회하면서 지도에 마커로 표시 & 마커에 커서 올리면 가게명 표시 기능
for i in range(len(df_no_duplicates)):
    lat = df_no_duplicates.loc[i, 'Latitude']
    long = df_no_duplicates.loc[i, 'Longitude']
    name = df_no_duplicates.loc[i,'가게명']
    folium.Marker([lat, long],tooltip= name).add_to(m)    


In [ ]:
m